Let's try implementing a simple datatype for wffs of propositional logic. We'll represent propositional variables as characters.

In [159]:
{-# LANGUAGE FlexibleContexts #-}


In [160]:
newtype Var = Var Char

instance Show Var where
    show (Var c) = [c]

Here is a recursive datatype for wffs of propositional logic. Essentially, anything of type `Expr` is a wff of propositional logic.

In [161]:
data Expr = Variable Var
    | Negation Expr
    | Conjunction Expr Expr
    | Disjunction Expr Expr
    | Conditional Expr Expr

We write a `Show` instance for our `Expr` datatype, to ensure that wffs are printed in a way that's readable.

In [185]:
parenthesise :: String -> String
parenthesise cs = "(" ++ cs ++ ")"

instance IHaskell.Display.IHaskellDisplay Expr where
    display (Variable (Var c)) = IHaskell.Display.display $ IHaskell.Display.latex ("$" ++ [c] ++ "$")

instance Show Expr where
    show (Variable v) = show v
    show (Negation f) = parenthesise $ "¬" ++ " " ++ show f
    show (Conjunction v v') = parenthesise $ show v ++ " ∧ " ++ show v'
    show (Disjunction v v') = parenthesise $ show v ++ " ∨ " ++ show v'
    show (Conditional v v') = parenthesise $ show v ++ " → " ++ show v'

We can even typeset wffs using `LaTeX`! 

In [186]:
(Variable (Var 'a'))

Line 1: Redundant bracket
Found:
(Variable (Var 'a'))
Why not:
Variable (Var 'a')